In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression


In [2]:
def simulate_gbm(Total_paths, steps_per_year, T, S0, mu, sigma):

    dt = 1/steps_per_year
    dW = np.random.normal(0, np.sqrt(dt),size=(Total_paths,steps_per_year*T)).T
    St = np.exp((mu-sigma ** 2 / 2) * dt + sigma * dW)
    St = np.vstack([np.ones(Total_paths),St])
    St[0] = S0
    St = St.cumprod(axis=0)
    return St


In [3]:
def func_boundary(alpha, strike, x, k):
    return (alpha[0] + alpha[3] * k - strike) + (alpha[1] + alpha[4] * k + 1) * x + (alpha[2] + alpha[5] * k) * x**2

def Dfunc_boundary(alpha, x, k):
    return  (alpha[1] + alpha[4] * k + 1) + 2 * (alpha[2] + alpha[5] * k) * x

def newtons_method(alpha, k, strike, initial_guess=35, epsilon=1e-8, max_iterations=1000):
    # Initial guess
    x_n = initial_guess
    
    for n in range(max_iterations):
        # Calculate f(x_n)
        fx_n =  func_boundary(alpha, strike, x_n, k)
        if abs(fx_n) < epsilon:
            #print('Found solution after',n,'iterations.')
            return x_n
        # Calculate f'(x_n) 
        fpx_n = Dfunc_boundary(alpha, x_n, k)
        
        if fpx_n == 0:  # Prevent division by zero
            print("Zero derivative. No solution found.")
            return None
        
        # Newton's update
        x_n = x_n - fx_n / fpx_n
        
    # If no convergence, return None
    print("Max iterations exceeded, no convergence.")
    return None


## Without path weights

In [6]:

# Parameter
# number of exercise dates per year
M = 50
# total number of paths
N = 100000
# number of iterations
n = 100
# time in years
T = 1
# initial stock price
S0 = 36
# volatility
sigma = 0.2
# strike
strike = 40
# let the annual risk-free interest rate
r = 0.06


# discounted price for all iterations
Price = [0]*n
# path info from iteration 1 to current iteration - a dictionary with k as the key
X = dict()
Y = dict()
# model info for each time step - a dictionary with k as the key
alphas = dict()


# initializaiton:
paths = simulate_gbm(int(N/n), M, T, S0, r, sigma)
euro_discounted_prices = np.ones_like(paths) * np.exp(-r/M)
final_payoff = strike - paths[-1]
euro_discounted_prices[0] = np.where(final_payoff > 0, final_payoff, 0)
euro_discounted_prices = euro_discounted_prices.cumprod(axis=0)
euro_discounted_prices = euro_discounted_prices[::-1]
for k in range(1, T*M+1):
    x1 = paths[k]
    x2 = x1*x1
    x3 = k*np.ones_like(x1)
    x4 = k*x1
    x5 = k*x2
    xs = np.column_stack([x1,x2,x3,x4,x5])

    if k==T*M:
        y = euro_discounted_prices[k]
    else:
        y = euro_discounted_prices[k+1] * np.exp(-r/M)
    model_sklearn = LinearRegression()
    model = model_sklearn.fit(xs, y)
    alphas[k] = np.append(np.array([model.intercept_]),model.coef_)


# iterations
for i in range(1, n+1):
    # for one interation
    # step 1: paths simulation
    paths =simulate_gbm(int(N/n), M, T, S0, r, sigma)

    # step 2.1: payoff value calculation
    payoff_values = strike - paths
    payoff_values[payoff_values<0] = 0
    # step 2.2: continuation value estimate:
    continuation_values = np.zeros_like(paths)
    for k in range(1, T*M+1):
        # in each time step, calculate the continuation values for different paths
        # using corresponding regression model(alpha_k)
        x0 = np.ones_like(paths[k]) # intercept
        x1 = paths[k]
        x2 = x1 * x1
        x3 = k * np.ones_like(x1)
        x4 = k * x1
        x5 = k * x2
        xs = np.column_stack([x0, x1, x2, x3, x4, x5])
        continuation_values[k]=  xs.dot(alphas[k])

    # step 3: comparison between continuation value and exrecise value
    discounted_price = payoff_values
    for k in range(T*M-1, 0, -1):
        no_exercise = discounted_price[k] < continuation_values[k]
        discounted_price[k][no_exercise] = discounted_price[k+1][no_exercise] * np.exp(-r/M)

    # step 4: sum of t=1 prices
    Price[i-1]= np.exp(-r/M) * discounted_price[1].sum()

    # step 5:update the regression models
    for k in range(1, T*M+1):
        x1 = paths[k]
        x2 = x1 * x1
        x3 = k * np.ones_like(x1)
        x4 = k * x1
        x5 = k * x2
        xs = np.column_stack([x1, x2, x3, x4, x5])

        if k not in X:
            X[k] = xs
        else:
            X[k] = np.vstack([X[k], xs])
        if k not in Y:
            if k == T*M:
                Y[k] = discounted_price[k]
            else:
                Y[k] = discounted_price[k + 1] * np.exp(-r / M)
        else:
            if k == T*M:
                Y[k] = np.hstack([Y[k], discounted_price[k]])
            else:
                Y[k] = np.hstack([Y[k], discounted_price[k + 1] * np.exp(-r / M)])

        model_sklearn = LinearRegression()
        model = model_sklearn.fit(X[k], Y[k])
        alphas[k] = np.append(np.array([model.intercept_]),model.coef_)

# weights given to different iterations
W = np.array([0.99*(i-1) for i in range(1,n+1)])
W = np.tanh(W)
W = 1 - 0.5 * (1 - W)

Price = np.array(Price) / (N/n)
Option_price = sum(Price * W) / W.sum()

print(Option_price, Price)


4.389376163267838 [4.1154476  4.42351481 4.35637337 4.43786818 4.20136315 4.27919113
 4.40342382 4.36680499 4.24818975 4.36548941 4.45403185 4.42420828
 4.48816953 4.40518167 4.39742684 4.48504735 4.44855462 4.36758483
 4.35541138 4.44835699 4.48398065 4.49550427 4.47175091 4.44503563
 4.4296101  4.44070245 4.31872897 4.42598394 4.41618729 4.20405595
 4.42239189 4.35312676 4.40243114 4.54590897 4.30551431 4.33494358
 4.3985536  4.45375511 4.52321246 4.46259469 4.25247449 4.16206614
 4.45398808 4.48631123 4.3497034  4.35844977 4.50949739 4.18036441
 4.25464245 4.31618744 4.31142491 4.44043862 4.38211667 4.5063607
 4.39469729 4.35709063 4.34199897 4.43680256 4.23443699 4.43697321
 4.42871701 4.36454546 4.32714702 4.62178302 4.3724082  4.41220649
 4.39081652 4.41329733 4.33233077 4.47471447 4.25537509 4.46928243
 4.35342268 4.30442956 4.2958999  4.35178267 4.24633398 4.50710669
 4.46302243 4.55605731 4.46198951 4.44923057 4.27041802 4.31242987
 4.3507605  4.45252952 4.41863179 4.47142116 

## with weights y_k and w_i

In [8]:
    lbd = 1.2 # lambda
    mu = 2 # mu

    # Parameter
    # number of excersie dates per year
    M = 50
    # total number of paths
    N = 100000
    # number of iterations
    n = 100
    # time in years
    T = 1
    # initial stock price
    S0 = 36
    # volatility
    sigma = 0.2
    # strike
    strike = 40
    # let the annual risk-free interest rate
    r = 0.06
    
    

    # discounted price for all iterations - a list
    Price = [0]*n
    # path info from iteration 1 to current iteration - a dictionary with k as the key
    X = dict()
    Y = dict()
    # model info for each time step - a dictionary
    alphas = dict()
    # initial exercise boundary
    exercise_boundary = 35
    
    # initializaiton:
    paths = simulate_gbm(int(N/n), M, T, S0, r, sigma)
    euro_discounted_prices = np.ones_like(paths) * np.exp(-r/M)
    final_payoff = strike - paths[-1]
    euro_discounted_prices[0] = np.where(final_payoff > 0, final_payoff, 0)
    euro_discounted_prices = euro_discounted_prices.cumprod(axis=0)
    euro_discounted_prices = euro_discounted_prices[::-1]
    for k in range(1, T * M + 1):
        x1 = paths[k]
        x2 = x1 * x1
        x3 = k * np.ones_like(x1)
        x4 = k * x1
        x5 = k * x2
        xs = np.column_stack([x1,x2,x3,x4,x5])
    
        if k == T * M:
            y = euro_discounted_prices[k]
        else:
            y = euro_discounted_prices[k+1] * np.exp(-r/M)
        model_sklearn = LinearRegression()
        model = model_sklearn.fit(xs, y)
        alphas[k] = np.append(np.array([model.intercept_]),model.coef_)
    
    # weights for paths
    w_i = np.array([])
    w_UV = [1 - lbd * np.exp(- i / mu) for i in range(1,n+1)]   
    
    # iterations
    for i in range(1,n+1):
        # for one interation
        # step 1:paths simulation
        paths =simulate_gbm(int(N/n), M, T, S0, r, sigma)
    
        # step 2.1: payoff value calculation
        payoff_values = strike - paths
        payoff_values[payoff_values<0] = 0
        # step 2.2: continuation value estimate:
        continuation_values = np.zeros_like(paths)
        for k in range(1,T*M+1):
            # in each time step, calculate the continuation values for different paths
            # using corresponding regression model(alpha_k)
            x0 = np.ones_like(paths[k]) # intercept
            x1 = paths[k]
            x2 = x1*x1
            x3 = k * np.ones_like(x1)
            x4 = k * x1
            x5 = k * x2
            xs = np.column_stack([x0, x1, x2, x3, x4, x5])
            continuation_values[k]=  xs.dot(alphas[k])
    
        # step 3: comparison between continuation value and exrecise value
        discounted_price = payoff_values
        for k in range(T*M-1,0,-1):
            no_exercise = discounted_price[k] < continuation_values[k]
            discounted_price[k][no_exercise]= discounted_price[k+1][no_exercise] * np.exp(-r/M)
    
        # step 4: sum of t=1 prices
        Price[i-1]= np.exp(-r/M) * discounted_price[1].sum()
    
        # step 5: update the regression models
        w_i = np.append(w_i, np.array([w_UV[i-1] ** (10 - i)] * int(N/n)))
        
        for k in range(1,T*M+1):
            x1 = paths[k]
            x2 = x1 * x1
            x3 = k * np.ones_like(x1)
            x4 = k * x1
            x5 = k * x2
            xs = np.column_stack([x1, x2, x3, x4, x5])
    
            if k not in X:
                X[k] = xs
            else:
                X[k] = np.vstack([X[k], xs])
            if k not in Y:
                if k == T*M:
                    Y[k] = discounted_price[k]
                else:
                    Y[k] = discounted_price[k + 1] * np.exp(-r / M)
            else:
                if k == T*M:
                    Y[k] = np.hstack([Y[k], discounted_price[k]])
                else:
                    Y[k] = np.hstack([Y[k], discounted_price[k + 1] * np.exp(-r / M)])
         
            root = newtons_method(alphas[k], k, strike)
            if root:
                exercise_boundary = root

            x = X[k][:,0] 
            beta_k = x.std()
            y_k = np.exp(- (abs(x - exercise_boundary))**2 / (2 * beta_k**2))
            w_k = w_i * y_k
            w_k = w_k * (w_k>0)
            model_sklearn = LinearRegression()
            model = model_sklearn.fit(X[k], Y[k], sample_weight = w_k)
            alphas[k] = np.append(np.array([model.intercept_]),model.coef_)
    
    # weights given to different iterations
    W = np.array([0.99*(i-1) for i in range(1,n+1)])
    W = np.tanh(W)
    W = 1 - 0.5 * (1 - W)
    
    Price = np.array(Price) / (N/n)
    Option_price = sum(Price * W) / W.sum()
    
    print(Option_price, Price)


Max iterations exceeded, no convergence.
Max iterations exceeded, no convergence.
Max iterations exceeded, no convergence.
Max iterations exceeded, no convergence.
Max iterations exceeded, no convergence.
Max iterations exceeded, no convergence.
Max iterations exceeded, no convergence.
Max iterations exceeded, no convergence.
4.3588032650780235 [4.32605638 4.36514196 4.50337787 4.30082629 4.40208205 4.3096103
 4.2058574  4.37144898 4.23309127 4.42654815 4.27545911 4.33228638
 4.55636424 4.34901836 4.13168127 4.50013853 4.32215434 4.28752162
 4.23916525 4.35354956 4.39816409 4.22396619 4.3921499  4.37070267
 4.4317956  4.35900938 4.37815162 4.23134315 4.36527298 4.24856699
 4.15066974 4.40887189 4.51337714 4.43492062 4.45760974 4.39617181
 4.46891703 4.3471313  4.2799982  4.30761344 4.36033003 4.30738249
 4.33910226 4.54411544 4.44908433 4.34739087 4.15324921 4.41514191
 4.38529739 4.4062725  4.32445204 4.15145063 4.50576081 4.41730265
 4.40110433 4.42616178 4.47593901 4.23080381 4.4264